#  Juliaで高速なベクトル・行列の区間演算をしたい

ベクトルの内積、行列ベクトル積、行列積の区間演算はとても重要。すなわち
$x, y\in\mathbb{IF}^n$, $A, B\in\mathbb{IF}^{n\times n}$ に対して、それぞれ$x^Ty \subset \alpha$, $Ax\subset z$, $AB\subset C$となる$\alpha\in\mathbb{IF}$, $z\in\mathbb{IF}^n$, $C\in\mathbb{IF}^{n\times n}$を求める方法を考える。

## 素朴な方法

最も素朴な方法は各演算を区間演算に変更した方式である。

$$
    \alpha = \sum_{k = 1}^n x_ky_k
$$

$$
    z_i = \sum_{k = 1}^n A_{i,k}x_k\quad (1\le i\le n)
$$

$$
    C_{ij} = \sum_{k+1}^n A_{i,k}B_{k,j}\quad (1\le i,j\le n)
$$

特にJuliaでは、計算を関数で包む事で、高速な計算ができることがあるらしい。


In [37]:
using IntervalArithmetic, BenchmarkTools

n = 1000;
x = randn(n);
y = randn(n);

x_int = map(Interval, x);
y_int = map(Interval, y);

@btime alpha = dot(x_int,y_int)

function dot_vec()
    n = 1000; x = randn(n); y = randn(n);
    x_int = map(Interval, x); y_int = map(Interval, y);
    alpha = dot(x_int,y_int);
end
@btime dot_vec()

  70.605 μs (6007 allocations: 93.88 KiB)
  79.435 μs (6011 allocations: 141.25 KiB)


[20.6663, 20.6664]

In [40]:
# function interval_gemv()
n = 1000; A = randn(n,n); x = randn(n);
A_int = map(Interval, A); x_int = map(Interval, x);
@btime z = A_int*x_int;
#     return z
# end

  39.466 ms (1 allocation: 15.75 KiB)


In [41]:
n = 1000; A = randn(n,n); B = randn(n,n);
A_int = map(Interval, A); B_int = map(Interval, B);
@btime C = A_int*B_int;

  41.813 s (8 allocations: 15.26 MiB)


これを高速化したい。考えられる方法は

- 並列化したい（[Loopvectrization](https://github.com/chriselrod/LoopVectorization.jl), [Tullio](https://github.com/mcabbott/Tullio.jl)?, @simd命令）
- [StaticArrays](https://github.com/JuliaArrays/StaticArrays.jl_)を使うと[速いらしい](https://qiita.com/cometscome_phys/items/669273a49ab2417d3af8)
- BLASを使う、だけど丸め方向が変えられないので、丸めの向きを変えない区間演算
    - 内積は森倉定理8あるいは教科書の定理2.4(p.38)
    - 行列ベクトル積は？
    - 行列積は森倉4節
 

In [42]:
using LoopVectorization, LinearAlgebra, StructArrays, BenchmarkTools, Test
using IntervalArithmetic
BLAS.set_num_threads(8);
# function mul_avx!(C::StructArray{Interval{Float64},2},
#                   A::StructArray{Interval{Float64},2},
#                   B::StructArray{Interval{Float64},2})
#     @avx for m ∈ 1:size(A,1), n ∈ 1:size(B,2)
#             Cmn = zero(eltype(C))
#             for k ∈ 1:size(A,2)
#                 Cmn += A[m,k] * B[k,n]
#             end
#             C[m,n] = Cmn
#          end
# end

function mul_avx!(C, A, B)
           @avx for m ∈ axes(A,1), n ∈ axes(B,2)
               Cmn = zero(eltype(C))
               for k ∈ axes(A,2)
                   Cmn += A[m,k] * B[k,n]
               end
               C[m,n] = Cmn
           end
       end

n = 500;
# A = StructArray(map(Interval,randn(n,n)));
# B = StructArray(map(Interval,randn(n,n)));
# C = StructArray(map(Interval,zeros(n,n)));
setrounding(Interval, :fast)
A = map(Interval,randn(n,n));
B = map(Interval,randn(n,n));
# C = map(Interval,zeros(n,n));
# A = randn(n,n);
# B = randn(n,n);
# C = zeros(n,n);
C = similar(A);

@btime mul_avx!($C, $A, $B)
@btime C1 = A*B;
# @test C1 ⊆ C

# using Gaius
# @btime blocked_mul!(C2, A, B)

  4.364 s (0 allocations: 0 bytes)
  4.155 s (8 allocations: 3.82 MiB)
